<h1> Stability of the Italian Government </h1>
<h3> Davide Cremonini </h3> 
<h3>Foundamentals of Artificial Intelligence - Module 3 Project </h3>

<p>This project aims at the definition of a Bayesian Network which captures the causal links between purely statistichal data of past Italian governments in an attempt to predict instability conditions for the future ones.</p>

In [53]:
#First we import every library which will be used
import pandas as pd
import matplotlib.pyplot as plt
from pgmpy.models import BayesianNetwork
import networkx as nx
from pgmpy.inference import VariableElimination, BeliefPropagation

In [54]:
#Reading and showing the dataset, performing NaN rows cleanup
df = pd.read_csv("Dataset_Governi_OK.csv", sep=';', decimal=',', true_values=['T', 'P'], false_values=['F'])
df=df.dropna(axis=0, how='all')
df.head()

,Gov Name,Time in Charge,Effective Time in Charge,House Majority,House Opposition,Senate Majority,Senate Opposition,N° of VP,N° Ministers,N° Ministers without portfolio,...,First Year,Legislation,Confidence,Fall Reason,PM from Largest Majority Party,Technocrat/Independent PM,N° Majority Parties,N° Opposition Parties,DI Parties in Government,Tot Gov. Members
0,De Gasperi V,613.0,598.0,369.0,205.0,156.0,237.0,3.0,17.0,2.0,...,1948.0,1.0,True,External Pressure,True,False,5.0,5.0,0.65,23.0
1,De Gasperi VI,545.0,535.0,336.0,238.0,149.0,88.0,0.0,17.0,3.0,...,1950.0,1.0,True,Internal Struggle,True,False,4.0,7.0,0.48,21.0
2,De Gasperi VII,721.0,704.0,317.0,257.0,139.0,98.0,1.0,17.0,2.0,...,1951.0,1.0,True,End of Legislation,True,False,3.0,7.0,0.26,21.0
3,De Gasperi VIII,32.0,12.0,263.0,327.0,116.0,12.0,1.0,16.0,2.0,...,1953.0,2.0,False,No confidence,True,False,1.0,8.0,0.00,20.0
4,Pella,155.0,141.0,324.0,266.0,141.0,102.0,0.0,16.0,2.0,...,1953.0,2.0,True,Internal Struggle,True,False,5.0,4.0,0.00,19.0


<h4>The dataset</h4>
<div>The dataset contains data on all of the 64 governments that lead Italy from the First Legislation to the Eighteenth.
This dataset was built by myself using the information found on the following sitse:<br>
    https://www.governo.it/it/i-governi-dal-1943-ad-oggi/191<br>
    https://www.senato.it/legislature/repubblica/governi-della-repubblica<br>
    https://it.wikipedia.org/wiki/Template:Governi_della_Repubblica_Italiana<br>
</div><br>
<div>
There are a total of 26 columns in the dataset:
<ol>
    <li><b>Gov Name</b>, the official government name, which derives from the surname of the Prime Minister.</li>
    <li><b>Time in Charge</b>, the number of days the government officially held power from its oath of office to the next one.</li>
    <li><b>Effective Time in Charge</b>, the number of days the government was actually in charge, from its oath of office to the resignation of the Prime Minister</li>
    <li><b>House Majority</b>, the number of House of Representatives voters which gave confidence to the government at the beginning of its office.</li>
    <li><b>House Opposition</b>, the number of House of Representatives voters which denied confidence or abstained to the government at the beginning of its office.</li>
    <li><b>Senate Majority</b>, the number of Senate of the Republic voters which gave confidence to the government at the beginning of its office.</li>
    <li><b>Senate Opposition</b>, the number of Senate of the Republic voters which denied confidence or abstained to the government at the beginning of its office.</li>
    <li><b>N° of VP</b>, the number of Vice-Prime Ministers each government had, it's mostly an honorific title given to the leaders of a lesser coalition party.</li>
    <li><b>N° of Ministers</b>, the number of Ministers each government had, they are usually present in every government, regardless of the political agenda.</li>
    <li><b>N° of Ministers without portfolio</b>, the number of Ministers without Portfolio each government had, theier roles change in every government as they are an expression of the political agenda of said government.</li>
    <li><b>Swapped Ministers</b>, the number of times a minister was changed during the office, this may have happened for different reasons, like resignation or death of said minister.</li>
    <li><b>DI House Majority</b>, the majority's diversity in the House of Representatives when the government asked for confidence. It is expressed according to Simpson's Diversity index, which takes into consideration each party as a different species and the number of voters it contained as the frequency for each species.</li>
    <li><b>DI House Opposition</b>, the opposition's diversity in the House of the Representatives, according to the SDI.</li>
    <li><b>DI Senate Majority</b>, the majority's diversity in the Senate of the Republic, according to the SDI.</li>
    <li><b>DI Senate Opposition</b>, the opposition's diversity in the Senate of the Republic, according to the SDI.</li>
    <li><b>End of the Legislation</b>, a boolean value which expresses whether or not one government was the last of its legislation.</li>
    <li><b>First Year</b>, the year number in which the government swore office.</li>
    <li><b>Legislation</b>, the number of parliamentary legislation which was present when the governement took office.</li>
    <li><b>Confidence</b>, a boolean which expresses whether or not each government gained confidence from both houses of the Parliament.</li>
    <li><b>Fall Reason</b>, a summarized short description which tries to explain which was the main cause for the government fall.</li>
    <li><b>PM from Largest Majority Party</b>, a boolean which expresses whether or not each government's PM was a member of the largest party of the coalition.</li>
    <li><b>Technocrat/Independent PM</b>, a boolean which expresses whether or not the Prime Minister of each government was not a member of any of parties inside the coalition (Independent) or, in case he was completely external from the political activity, a so called 'Technocrat PM'.</li>
    <li><b>N° Majority Parties</b>, the number of parties which mostly voted for giving confidence to the government.</li>
    <li><b>N° Opposition Parties</b>, the number of parties which denied or abstained from giving confidence to the government.</li>
    <li><b>DI Parties in Government</b>, party diversity inside the government seats, according to SDI using the parties as species and the numbers of PM, and Ministers as frequency.</li>
    <li><b>Tot Gov. Members</b>, total number of government members, a sum of values included in 'N° of VP', 'N° of Ministers' and 'N° of Ministers without Portfolio', plus one for the Prime Minister</li>
</ol>
</div>

In [55]:
#Pre-prorcessing Actvities

#Changing Majority and Opposition numbers in both Houses to a unique percentage for both.
percentages = []
for i in range(len(df)):
    tot_seats = df['House Majority'][i] + df['House Opposition'][i] + df['Senate Majority'][i] + df['Senate Opposition'][i] 
    perc_m = int(round((df['House Majority'].iloc[i]+df['Senate Majority'].iloc[i])/tot_seats,2)*100)
    perc_o = int(round((df['House Opposition'].iloc[i]+df['Senate Opposition'].iloc[i])/tot_seats,2)*100)
    percentages.append([perc_m, perc_o])
perc_df = pd.DataFrame(percentages, columns=['Tot % Majority','Tot % Opposition'])
df = pd.concat([df.drop(['House Majority', 'House Opposition', 'Senate Majority', 'Senate Opposition'], axis=1), perc_df], axis=1)

#Computing the average of SDI between both Houses for Majority and Opposition.
dis = []
for i in range(len(df)):
    di_m = int(round((df['DI House Majority'].iloc[i]+df['DI Senate Majority'].iloc[i])/2,2)*100)
    di_o = int(round((df['DI House Opposition'].iloc[i]+df['DI Senate Opposition'].iloc[i])/2,2)*100)
    dis.append([di_m, di_o])
di_df = pd.DataFrame(dis, columns=['Tot DI Majority','Tot DI Opposition'])
df = pd.concat([df.drop(['DI House Majority', 'DI House Opposition', 'DI Senate Majority', 'DI Senate Opposition'], axis=1), di_df], axis=1)

#Transforming the SDI of government members into a integer.
df['DI Parties in Government'] = 100*df['DI Parties in Government']
df['DI Parties in Government'] = df['DI Parties in Government'].astype(int)

#Dropping the columns which represent the number of Vice-PM, Ministers with and without portfolio as their sum is included in the Tot. Gov. Members column
df = df.drop(['N° of VP', 'N° Ministers', 'N° Ministers without portfolio'], axis=1)
df['Tot Gov. Members'] = df['Tot Gov. Members'].astype(int)

#Dropping non-relevant data
df = df.drop(['Gov Name', 'End of Legislation', 'Time in Charge'], axis=1)

<h4>Pre-processing Activities</h4>
<div>As the dataset was kept as much broad as possible while designing it, in order to be reused in future projects, we have to perform some pre-processing activities to make it suitable for the definition of the Bayesian Network using the pgmpy library. In order to do so we have perform some initial simplification before we can start with the encoding part. This simplification will be necessary to create a more compact and easy to analyse model, at che cost of disregarding some potential causal links but with the objective of letting others emerge.
<ul>
    <li>The first simplification was the redeuction of the majority and opposition votes, these were converted by summing the two numbers for house and senate for both and then expressing them through the integer percentage they each represent.</li>
    <li>Then, a similar procedure was applied to the SDIs for Majority and Opposition which were replaced by their respective macro average for both majority and opposition.</li>
    <li>The SDI for government composition was converted into an integer to ease further discretization. </li>
    <li>The number of government members, being the sum of other three columns, will only be used as a measure of the size of the government, so its components were be dropped.</li>
    <li> Government Name was dropped as it is merely a nominal index.</li>
    <li> End of Legislation was dropped as it is already included in Fall Reason.</li>
    <li> As we chose to consider the period of actual activity of each governement, for the sake of determining the stability measure, we'll focus only on the Effecive Time in Charge, so Time in Charge was dropped.</li>
</ul>
    
</div>

In [56]:
#Encoding

#Replacing all Booleans with 0 or 1
df = df.replace({True:1, False:0})


#Replacing Fall Reason with number encoding
nominal_replacements = {
    'External Pressure':'EP',
    'Internal Struggle':'IS',
    'End of Legislation':'EL',
    'No confidence':'NC',
    'Lost Confidence':'LC',
    'Reshuffle':'R'
}
df = df.replace(nominal_replacements)


#Encoding dictionary for non-boolean data
encode_dict = {
    'Effective Time in Charge': {'Intervals':[0,100,250,450,1000, 10000] ,'Labels':[0,1,2,3,4],},
    'Tot % Majority':{'Intervals':[0,50,55,66,100] ,'Labels':[0,1,2,3],},
    'Tot % Opposition':{'Intervals':[0,25,40,50,100] ,'Labels':[0,1,2,3],},
    'Tot DI Majority':{'Intervals':[-1,0,10,30,75,100] ,'Labels':[0,1,2,3,4],},
    'Tot DI Opposition':{'Intervals':[0,20,45,75,100] ,'Labels':[0,1,2,3],},
    'DI Parties in Government':{'Intervals':[-1,1,25,55,75,100] ,'Labels':[0,1,2,3,4],},
    'Tot Gov. Members':{'Intervals':[0,22,28,100] ,'Labels':[0,1,2],},
    'Swapped Ministers':{'Intervals':[-1,0,3,6,20] ,'Labels':[0,1,2,3],},
    'N° Majority Parties':{'Intervals':[0,1,4,15] ,'Labels':[0,1,2],},
    'N° Opposition Parties':{'Intervals':[0,1,4,15] ,'Labels':[0,1,2],},
    'First Year':{'Intervals':[0,1950, 1960,1970,1980, 1990,2000,2010,2020,2030] ,'Labels':[0,1,2,3,4,5,6,7,8],},
    'Legislation':{'Intervals':[0,12,50] ,'Labels':[0,1],},
}

for k in encode_dict.keys():
    #print(f"Encoding data {k} with intervals {encode_dict[k]['Intervals']} and labels:{encode_dict[k]['Labels']}")
    df[k] = pd.DataFrame(pd.cut(df[k].astype(int), encode_dict[k]['Intervals'], labels = encode_dict[k]['Labels']))

<h4>Encoding</h4>
<div>
Discretization is essential to highlight possible relationships between groups of values. In the case of the dataset presented each non-boolean value was reduced into a spceific set of ranges according to conventions or trying to give an ordered and rational separation of values in different categories.
<b>Effective Time in Charge:</b> 
    <ul>
    <li>Unstable (0 to 100) ---> 0 </li>
    <li>Weak stability (101 to 250) ---> 1 </li>
    <li>Average stability (251 to 450) ---> 2 </li>
    <li>Solid stabiliy (451 to 1000) ---> 3 </li>
    <li>High stability (Over 1000) ---> 4 </li>
    </ul>

<b>Percentatages of Majority:</b>
    <ul>
    <li>No majority (0 to 50) ---> 0 </li>
    <li>Weak majorty (51 to 55) ---> 1 </li>
    <li>Strong majority (56 to 66) ---> 2 </li>
    <li>Constitutional majority (Over 66) ---> 3</li>
    </ul>

<b>Percentages of Opposition:</b>
    <ul>
    <li>Weak opposition (0 to 25) ---> 0 </li>
    <li>Medium opposition (26 to 40) ---> 1 </li>
    <li>Strong opposition (41 to 50) ---> 2 </li>
    <li>Potential new majority Over 50) ---> 3</li>
    </ul>

<b>N° Majority Parties:</b>
    <ul>
    <li>Single Party Majority (Only 1) ---> 0 </li>
    <li>Coalition Majority (2 to 4) ---> 1 </li>
    <li>Broad Coalition Majority (Over 5) ---> 2 </li>
    </ul>

<b>N° Opposition Parties:</b>
    <ul>
    Single Party Opposition (Only 1) ---> 0  </li>
    Differentiated Opposition (2 to 4 ---> </li>
    Over 5 ---> Fragmented Opposition</li>
    </ul>

<h5>Distribution Index for Majority x100:</h5>
    <div>0 ---> Single party majority<br>
    1 to 10 ---> Nearly one sided majority<br>
    11 to 30 ---> Strong lead majority<br>
    31 to 75 ---> Fragmented majority<br>
    Over 76 ---> Multi-headed majority<br>
</div>

<h5>Distribution Index for Opposition x100:</h5>
    <div>0 to 20 ---> Cohesive Opposition<br>
    21 to 45 ---> Diverse Opposition<br>
    46 to 75 ---> Fragmented Opposition<br>
    Over 76 ---> Higly Fragmented/ Multi-Headed Opposition<br>
</div>

<h5>Distribution Index for Government Seats x100:</h5>
    <div>0 ---> Monochromatic Government<br>
    1 to 25 ---> Governement with Leading Party<br>
    26 to 55 ---> Coalition Government<br>
    56 to 75 ---> Large Coalition Government<br>
    Over 75 ---> Super Coalition/National Unity Government<br>
</div>

<h5>Members of the Government</h5>
<div> Under 22 ---> Small Government<br>
    23 to 28 ---> Medium Government<br>
    Over 29 ---> Large Government<br>
    
</div>

<h5>Swapped Ministers</h5>
<div> 0 ---> Unchanged<br>
    Under 3---> Small Changes<br>
    3 to 6 ---> Medium Changes<br>
    Over 6 ---> Large Changes<br>
    
</div>
</div>

In [57]:
#Building the Network

<h4>Building the Network</h4>

In [58]:
#Queries

<h4>Making some Queries</h4>

<h4>Conclusions</h4>